In [1]:
import cv2
import numpy as np
import time
from collections import deque
import img2pdf
import os
import glob

In [2]:

colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255)]
counter = 0
PATH = r"Downloads/"
# Define the upper and lower boundaries for a color to be considered "Blue" HSV


# Define a 5x5 kernel for erosion and dilation
kernel = np.ones((5, 5), np.uint8)

# Setup deques to store separate colors in separate arrays
bpoints = [deque(maxlen=512)]
gpoints = [deque(maxlen=512)]
rpoints = [deque(maxlen=512)]


# for contanst time insertion
bindex = 0
gindex = 0
rindex = 0

colorIndex = 0     # Active Color

# check boxes

t1 = False               # Default
t2 = True
t3 = False

# Bakcground

switch = True  # True for Black

# Thickness

th = 3


In [3]:
def setValues(x):
    pass
    
cv2.namedWindow("Color detectors")
cv2.createTrackbar("Upper Hue","Color detectors", 153,180,setValues)
cv2.createTrackbar("Upper Saturation","Color detectors", 255,255,setValues)
cv2.createTrackbar("Upper Value","Color detectors", 255,255,setValues)
cv2.createTrackbar("Lower Hue","Color detectors", 64,180,setValues)
cv2.createTrackbar("Lower Saturation","Color detectors", 171,255,setValues)
cv2.createTrackbar("Lower Value","Color detectors", 78,255,setValues)

camera = cv2.VideoCapture(0)
def make_720p():
    camera.set(3, 1280)
    camera.set(4, 720)
    
make_720p()

while True:
    
    
    grabbed, frame = camera.read()
    frame = cv2.resize(frame,(900,700))
    frame = cv2.flip(frame, 1)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
   
    notebook = np.zeros((590,660,3)) + 255
    if switch == True:
        notebook = np.zeros((590,660,3))
    
    frame = cv2.rectangle(frame, (300,70), (850,600), (0,0,0), 3)
    
    
    if not grabbed :
        print("Error loading video!")
        break
     
# ****************************** UI *****************************************************************************

    
        
        
    frame = cv2.rectangle(frame, (300,620), (370,690), (215,215,220), -1) # Eraser
    frame = cv2.rectangle(frame, (420,620), (490,690), colors[0], -1)     # Blue
    frame = cv2.rectangle(frame, (540,620), (610,690), colors[1], -1)     # Green
    frame = cv2.rectangle(frame, (660,620), (730,690), colors[2], -1)     # Red
    frame = cv2.rectangle(frame, (300,10), (450,60), (156,26,43), -1)     # Download
    
    frame = cv2.rectangle(frame, (600,10), (602,60), (85,79,80), -1)     # thickness 1
    frame = cv2.rectangle(frame, (700,10), (706,60), (85,79,80), -1)   # thickness 2
    frame = cv2.rectangle(frame, (800,10), (808,60), (85,79,80), -1)   # thickness 3
    
    
    
    if t1 == True:
         frame = cv2.rectangle(frame, (560,30), (590,60), (255,255,255), -1)
            
    
    
    if t2 == True:
         frame = cv2.rectangle(frame, (660,30), (690,60), (255,255,255), -1)
     
   
    
    if t3 == True:
         frame = cv2.rectangle(frame, (760,30), (790,60), (255,255,255), -1)
    
    frame = cv2.rectangle(frame, (560,30), (590,60), (0,0,0), 2)   # check boxes
    frame = cv2.rectangle(frame, (760,30), (790,60), (0,0,0), 2)   # check boxes
    frame = cv2.rectangle(frame, (660,30), (690,60), (0,0,0), 2)   # check boxes
    
   

    
    frame = cv2.circle(frame, (int(800), int(655)), int(50), (215,215,220), -1) # Clear
    frame = cv2.circle(frame, (int(800), int(655)), int(50), (0,0,0), 2) # Clear
    
    frame = cv2.rectangle(frame, (300,620), (370,690), (0,0,0), 2) # Eraser
    frame = cv2.rectangle(frame, (420,620), (490,690), (0,0,0), 2)     # Blue
    frame = cv2.rectangle(frame, (540,620), (610,690), (0,0,0), 2)     # Green
    frame = cv2.rectangle(frame, (660,620), (730,690), (0,0,0), 2)     # Red
    frame = cv2.rectangle(frame, (300,10), (450,60), (0,0,0), 2)     # Download    
    frame = cv2.rectangle(frame, (600,10), (602,60), (0,0,0), 2)   # thickness 1
    frame = cv2.rectangle(frame, (700,10), (706,60), (0,0,0), 2)   # thickness 2
    frame = cv2.rectangle(frame, (800,10), (806,60), (0,0,0), 2)   # thickness 3
    
    
        
    cv2.putText(frame, "Eraser", (301, 648), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, "Blue", (425, 648), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, "Green", (545, 648), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, "Red", (665, 648), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, "Clear", (765, 655), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, "DOWNLOAD", (305, 38), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)
    
# ********************************* Morphology *******************************************************************

    # Determine which pixels fall within the blue boundaries and then blur the binary image
    
    u_hue = cv2.getTrackbarPos("Upper Hue","Color detectors")
    u_sat = cv2.getTrackbarPos("Upper Saturation","Color detectors")
    u_value = cv2.getTrackbarPos("Upper Value","Color detectors")
    l_hue = cv2.getTrackbarPos("Lower Hue","Color detectors")
    l_sat = cv2.getTrackbarPos("Lower Saturation","Color detectors")
    l_value = cv2.getTrackbarPos("Lower Value","Color detectors")
    blueUpper = np.array([u_hue,u_sat,u_value])
    blueLower = np.array([l_hue,l_sat,l_value])
    blueMask = cv2.inRange(hsv, blueLower, blueUpper)
    #blueMask = cv2.erode(blueMask, kernel, iterations=2)
    blueMask = cv2.morphologyEx(blueMask, cv2.MORPH_OPEN, kernel)
    

    # Find contours in the image
    cnts, _ = cv2.findContours(blueMask.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    center = None
    

    # Check to see if any contours were found
    if len(cnts) > 0:
        
    	# Sort the contours and find the largest one 
    	
        cnt = sorted(cnts, key = cv2.contourArea, reverse = True)[0]
        
        # Get the radius of the enclosing circle around the found contour
        
        ((x, y), radius) = cv2.minEnclosingCircle(cnt)
        
        # Draw the circle around the contour
        #cv2.circle(frame, (int(x), int(y)), int(radius), (0, 255, 255), 2)
        # Get the moments to calculate the center of the contour (in this case Circle)
        
        M = cv2.moments(cnt)
        center = (int(M['m10'] / M['m00']), int(M['m01'] / M['m00']))
       
        val = list(center)
        
        # Logic
        if center[0]  > 300:
            
            cv2.circle(frame, (int(center[0]), int(center[1])), int(7), colors[colorIndex], -1)
            if switch:
                cv2.circle(notebook, (int(center[0]-280), int(center[1]-50)), int(7), (255,255,255), -1)
            else:
                cv2.circle(notebook, (int(center[0]-280), int(center[1]-50)), int(7), (0,0,0), -1)
                
            if center[1] < 70:
                
                
                        
                if center[0]>= 560 and center[0] <= 590:
                    t1 = True
                    t2 = False
                    t3 = False
                    
                    bpoints.append(deque(maxlen=512))
                    bindex += 1
                    gpoints.append(deque(maxlen=512))
                    gindex += 1
                    rpoints.append(deque(maxlen=512))
                    rindex += 1
                    
                    th = 2
                    
                elif center[0] >= 660 and center[0] <= 690:
                    t1 = False
                    t2 = True
                    t3 = False
                    
                    
                    bpoints.append(deque(maxlen=512))
                    bindex += 1
                    gpoints.append(deque(maxlen=512))
                    gindex += 1
                    rpoints.append(deque(maxlen=512))
                    rindex += 1
                    
                    th = 3
                    
                elif center[0] >= 760 and center[0] <= 790:
                    t1 = False
                    t2 = False
                    t3 = True
                    th = 6 
                    bpoints.append(deque(maxlen=512))
                    bindex += 1
                    gpoints.append(deque(maxlen=512))
                    gindex += 1
                    rpoints.append(deque(maxlen=512))
                    rindex += 1
                   
            
                elif center[0] >= 300 and center[0] <= 450:
                    flag = False
                    points = [bpoints, gpoints, rpoints]
                    for i in range(len(points)):
                        for j in range(len(points[i])):
                            for k in range(1, len(points[i][j])):
                
                                if points[i][j][k - 1] is None or points[i][j][k] is None:
                                    continue 
                                flag = True
                                x1 = points[i][j][k][0]
                                y1 = points[i][j][k][1]
                                tck = points[i][j][k][2]
                                x2 = points[i][j][k-1][0]
                                y2 = points[i][j][k-1][1]
                                
                                #cv2.line(frame, points[i][j][k-1], points[i][j][k], colors[i], 2)
                                cv2.line(notebook, (x1 - 280, y1 - 50),(x2 - 280, y2 - 50), colors[i],tck)
                    
                    notebook = cv2.GaussianBlur(notebook,(3,3),0)
                    
                    
                    if flag:
                            
                        here = PATH+ str(counter) +".jpg"
                        down = cv2.resize(notebook,(1030,500))
                        cv2.imwrite(here, down)
                        counter += 1
                
                        cv2.putText(frame, "Download Successfull !", (390, 300), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                        time.sleep(0.5)
                    else:
                         cv2.putText(frame, "Empty notebook !", (390, 300), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

                
            elif center[1] > 600 :
                
                if center[0] > 300 and center[0] < 370:     # Eraser
                    
                    if colorIndex == 0:
                        bpoints = [deque(maxlen=512)]
                        bindex = 0 
                    elif colorIndex == 1:
                        gpoints = [deque(maxlen=512)]
                        gindex = 0
                    elif colorIndex == 2:
                        rpoints = [deque(maxlen=512)]
                        rindex = 0
                        
                elif center[0] > 420 and center[0] < 490:
                    colorIndex = 0 # B

                elif center[0] > 540 and center[0] < 610:
                    colorIndex = 1 # G

                elif  center[0] > 660 and center[0] < 730:
                    colorIndex = 2 # R

                elif center[0] > 750 and center[0] < 850:

                    bpoints = [deque(maxlen=512)]
                    gpoints = [deque(maxlen=512)]
                    rpoints = [deque(maxlen=512)]
                    


                    bindex = 0
                    gindex = 0
                    rindex = 0
                  
                    if switch:
                        notebook[:,:,:] = 0
                    else:
                        notebook[:,:,:] = 255
                
            else:
                val.append(th)
                if colorIndex == 0:
                    bpoints[bindex].appendleft(val)
                elif colorIndex == 1:
                    gpoints[gindex].appendleft(val)
                elif colorIndex == 2:
                    rpoints[rindex].appendleft(val)
                
              
        else:
            if bindex > 1000 or rindex > 1000 or gindex > 1000:
                print("Memory full !")
                break

            bpoints.append(deque(maxlen=512))
            bindex += 1
            gpoints.append(deque(maxlen=512))
            gindex += 1
            rpoints.append(deque(maxlen=512))
            rindex += 1
           
    else:
        if bindex > 1000 or rindex > 1000 or gindex > 1000:
            print("Memory full !")
            break
            
        bpoints.append(deque(maxlen=512))
        bindex += 1
        gpoints.append(deque(maxlen=512))
        gindex += 1
        rpoints.append(deque(maxlen=512))
        rindex += 1

            

    points = [bpoints, gpoints, rpoints]
    for i in range(len(points)):
        for j in range(len(points[i])):
            for k in range(1, len(points[i][j])):
                
                if points[i][j][k - 1] is None or points[i][j][k] is None:
                    continue 
                x1 = points[i][j][k][0]
                y1 = points[i][j][k][1]
                tck = points[i][j][k][2]
                x2 = points[i][j][k-1][0]
                y2 = points[i][j][k-1][1]
                cv2.line(frame,  (x1 , y1 ),(x2 , y2 ), colors[i],tck)
                
                cv2.line(notebook, (x1 - 280, y1 - 50),(x2 - 280, y2 - 50), colors[i],tck)
    
    
                
    
#************************************** END *******************************************
    
    cv2.imshow('UI', frame)
    cv2.imshow('Notebook',notebook)
    
    #cv2.imshow("Blue mask", blueMask)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        cv2.putText(frame, "Processing Notebooks !", (390, 300), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        with open("RJ11_B1_SWE.pdf","wb") as f:
            f.write(img2pdf.convert(glob.glob(r"Downloads/*.jpg")))
            
        break
        
  
    
        
camera.release()
cv2.destroyAllWindows()

### 